# State-of-the-art `vorongen` workflow

Pure import-driven interaction with `vorongen` as a pip package.

In [ ]:
# Optional in notebook environments:
# %pip install -e ..[torch]

In [ ]:
from dataclasses import replace

from vorongen import (
    RunConfig,
    TorchControllerConfig,
    VorongenSynthesizer,
    get_sample_config,
    is_torch_available,
)

print("PyTorch available:", is_torch_available())

In [ ]:
config = get_sample_config("mixed")

run_cfg = RunConfig(
    n_rows=4000,
    seed=19,
    tolerance=0.04,
    max_attempts=2,
    log_level="quiet",
    collect_history=True,
    use_torch_controller=True,
    torch_required=False,
    optimize_overrides={
        "batch_size": 768,
        "max_iters": 90,
        "patience": 10,
    },
    torch_controller=TorchControllerConfig(
        enabled=True,
        lr=2e-3,
        hidden_dim=48,
        min_multiplier=0.02,
        max_multiplier=0.11,
    ),
)

engine = VorongenSynthesizer(config, run_cfg)
result = engine.generate()

In [ ]:
{
    "success": result.success,
    "attempts": result.attempts,
    "objective": result.metrics["objective"],
    "confidence": result.quality_report["confidence"],
    "runtime_notes": result.runtime_notes,
}

In [ ]:
result.dataframe.head()

In [ ]:
top_errors = sorted(
    result.quality_report["per_column"],
    key=lambda row: row["marginal_error"] + row["conditional_error"],
    reverse=True,
)[:5]
top_errors

In [ ]:
classic_cfg = replace(run_cfg, use_torch_controller=False)
classic_result = VorongenSynthesizer(config, classic_cfg).generate()

{
    "classic_objective": classic_result.metrics["objective"],
    "torch_objective": result.metrics["objective"],
    "classic_confidence": classic_result.quality_report["confidence"],
    "torch_confidence": result.quality_report["confidence"],
}